# Code created to cleanse the Affected Pop Data

The data for this script comes daily, and the incountry IM are providing it through signal. The main folder are:
1. Orginal data: *D:\\NextCloud\\Projects\\2023\\2023-per-001\\GIS\\1_Original_Data\\115_UNDAC\\Pop_Affect*
2. Final output: *D:\\NextCloud\\Projects\\2023\\2023-per-001\\GIS\\2_Active_Data\\203_affd*

<code style="background:yellow;color:black">This notebook only works for Lambayeque, Piura and Tumbe. To make it work for the whole country a couple more edits in the source data need to take place. Please speak to Luis if this needs doing</code>

<code style="background:yellow;color:black">Please make sure to complete the variables below before running the notebook.  The date for the data is as per the file name sent by UNDAC</code>

In [11]:
DATA_DATE = '2023_04_27'
FILE_LOCATION = '20230427\\RS Nro 181_NACIONAL - REPORTE DE DANOS LLUVIAS 2023_27ABR2023_MA.xlsx'

## 1. Import packages

In [12]:
import pandas as pd
import numpy as np
from pathlib import Path


In [13]:
## Setting Paths and Files
fldr_path = Path('D:\\NextCloud\\Projects\\2023\\2023-per-001\\GIS\\1_Original_Data\\115_UNDAC\\Pop_Affect')
fldr_outp = Path('D:\\NextCloud\\Projects\\2023\\2023-per-001\\GIS\\2_Active_Data\\203_affd')

file_path = Path(fldr_path / f'{FILE_LOCATION}')
file_pcod = Path(fldr_outp / 'per_pcode_reference_es.xlsx')
file_name = Path(fldr_outp / 'per_name_change_es.xlsx')


## Setting PCODES Dataframe


In [14]:
# 1. Setting the ADM2 PCODE information
df_pc = pd.read_excel(file_pcod, sheet_name='PCODES')
df_pc

,ADM3_ES,ADM3_PCODE,ADM2_ES,ADM2_PCODE,ADM1_ES,ADM1_PCODE
0,Abancay,PE030101,Abancay,PE0301,Apurimac,PE03
1,Abelardo Pardo Lezameta,PE020502,Bolognesi,PE0205,Ancash,PE02
2,Acari,PE040302,Caraveli,PE0403,Arequipa,PE04
3,Acas,PE021402,Ocros,PE0214,Ancash,PE02
4,Accha,PE081002,Paruro,PE0810,Cusco,PE08
...,...,...,...,...,...,...
1868,Zarumilla,PE240301,Zarumilla,PE2403,Tumbes,PE24
1869,Zepita,PE210407,Chucuito,PE2104,Puno,PE21
1870,Zorritos,PE240201,Contralmirante Villar,PE2402,Tumbes,PE24
1871,Zuñiga,PE150516,Cañete,PE1505,Lima,PE15


## Setting NAME dataframe
this is to have the right names as per the PCODE dataset

In [15]:
df_nm = pd.read_excel(file_name, sheet_name='NAMES')
df_nm

,OLD_NAME,ADM_TYPE,NEW_NAME
0,AMAZONAS,DEPA,AMAZONAS
1,BAGUA,PROV,BAGUA
2,ARAMANGO,DIST,ARAMANGO
3,COPALLIN,DIST,COPALLIN
4,EL PARCO,DIST,EL PARCO
...,...,...,...
1788,IRAZOLA,DIST,IRAZOLA
1789,NESHUYA,DIST,NESHUYA
1790,PADRE ABAD,DIST,PADRE ABAD
1791,PURUS,PROV,PURUS


## Working with the affected pop data

In [16]:
# Open excel file in a dataframe
df_pop = pd.read_excel(file_path, sheet_name='EVALUADOR', skiprows=8)

# Only workk with the wanted columns

## FOR HOUSING
df_pop = df_pop[['Unnamed: 0', 'VIVIENDA DESTRUIDA', 'VIVIENDA INHABITABLE', 'VIVIENDA AFECTADA']].copy()
col_names = ['ADM_NAME', 'HOUSE_DEST', 'HOUSE_INHB', 'HOUSE_AFFT'] 	

df_pop.columns = col_names
df_pop = df_pop.iloc[1:] # remove the total general tab

# Remove the extra text that isn't needed
# 1. Extract DEP, PROV and DIST
df_pop['ADM_TYPE'] = df_pop['ADM_NAME'].str.extract('([A-Za-z]+)')

# 2. Remove DIST. DEPA. PROV. part from string
df_pop = df_pop.replace(regex=['DIST.'], value='')
df_pop = df_pop.replace(regex=['PROV.'], value='')
df_pop = df_pop.replace(regex=['DEPA.'], value='')

# 3. Make sure that there aren't random spaces
df_pop['ADM_NAME'] = df_pop['ADM_NAME'].str.lstrip()

# 4. Correct the name PROVIDENCIA as this was modified in the previous step
df_pop.loc[df_pop['ADM_NAME'] == 'DENCIA', 'ADM_NAME'] = 'PROVIDENCIA'
df_pop['ADM_NAME'] = np.where(df_pop['ADM_NAME'] == 'DENCIA', 'PROVIDENCIA',df_pop['ADM_NAME'] )


# 5. Create the extra columns needed
df_pop['AMD1_ES'] = ''
df_pop['AMD2_ES'] = ''
df_pop['ADM3_PCODE'] = ''

df_pop

,ADM_NAME,HOUSE_DEST,HOUSE_INHB,HOUSE_AFFT,ADM_TYPE,AMD1_ES,AMD2_ES,ADM3_PCODE
1,AMAZONAS,27.0,190.0,533.0,DEPA,,,
2,BAGUA,10.0,58.0,192.0,PROV,,,
3,ARAMANGO,NaN,NaN,NaN,DIST,,,
4,BAGUA,NaN,NaN,NaN,DIST,,,
5,COPALLIN,1.0,4.0,13.0,DIST,,,
...,...,...,...,...,...,...,...,...
1809,IRAZOLA,2.0,92.0,452.0,DIST,,,
1810,NESHUYA,NaN,NaN,47.0,DIST,,,
1811,PADRE ABAD,7.0,9.0,764.0,DIST,,,
1812,PURUS,NaN,NaN,NaN,PROV,,,


In [17]:
# ADD DEP and PROV names to the right column
df_pop['AMD1_ES'] = np.where(df_pop['ADM_TYPE'] == 'DEPA', df_pop['ADM_NAME'],np.nan)
df_pop['AMD2_ES'] = np.where(df_pop['ADM_TYPE'] == 'PROV', df_pop['ADM_NAME'],np.nan)
df_pop

# Forward fill the DEP and PROV columns with the right names
df_pop['AMD1_ES'].fillna(method='ffill', inplace=True)
df_pop['AMD2_ES'].fillna(method='ffill', inplace=True)

# Make sure that DEPT does not have ADM2
df_pop['AMD2_ES'] = np.where(df_pop['ADM_TYPE'] == 'DEPA', '',df_pop['AMD2_ES'])

# Only work with the departmentos we are interested in
########### THIS IS THE STEP MENTIONED IN THE DESCRIPTION OF THE NOTEBOOK##############
df_pop = df_pop[df_pop.AMD1_ES.isin(['LAMBAYEQUE','PIURA','TUMBES'])]

# Remove DEPT as that isn't needed anymore
df_pop = df_pop[~df_pop['ADM_TYPE'].isin(['DEPA'])]

df_pop

,ADM_NAME,HOUSE_DEST,HOUSE_INHB,HOUSE_AFFT,ADM_TYPE,AMD1_ES,AMD2_ES,ADM3_PCODE
1230,CHICLAYO,221.0,1094.0,6706.0,PROV,LAMBAYEQUE,CHICLAYO,
1231,CAYALTI,3.0,3.0,213.0,DIST,LAMBAYEQUE,CHICLAYO,
1232,CHICLAYO,6.0,120.0,220.0,DIST,LAMBAYEQUE,CHICLAYO,
1233,CHONGOYAPE,NaN,60.0,462.0,DIST,LAMBAYEQUE,CHICLAYO,
1234,ETEN,NaN,1.0,51.0,DIST,LAMBAYEQUE,CHICLAYO,
...,...,...,...,...,...,...,...,...
1787,ZARUMILLA,12.0,53.0,607.0,PROV,TUMBES,ZARUMILLA,
1788,AGUAS VERDES,10.0,NaN,82.0,DIST,TUMBES,ZARUMILLA,
1789,MATAPALO,1.0,48.0,335.0,DIST,TUMBES,ZARUMILLA,
1790,PAPAYAL,1.0,NaN,102.0,DIST,TUMBES,ZARUMILLA,


## Creating the final file for admn 3

In [18]:
df_final = df_pop.copy()

# Loop through the data to find the right value
# We are looking for match PROVINCE - DISTRICT to avoid name errors
for index, row in df_final.iterrows():
    if row['ADM_TYPE'] == 'DIST':
        # create temp dataframe
        df_temp = pd.DataFrame()
        
        # read PROV and DISTRC names
        prov_name = row['AMD2_ES']
        dist_name = row['ADM_NAME']
        
        # FOR THE FOLLOWING DISTRICT WE DON'T HAVE ADMIN BOUNDARY
        if dist_name not in ['AHUAYRO', 'PUTIS', 'UNION PROGRESO', 
                             'RIO MAGDALENA', 'NINABAMBA', 'PATIBAMBA', 
                             'KUMPIRUSHIATO', 'CIELO PUNCO', 'MANITEA', 
                             'UNION ASHANINKA', 'LAMBRAS', 'COCHABAMBA', 
                             'SAN ANTONIO', 'SANTA LUCIA', 'HUIPOCA','BOQUERON'] :
            
            
            # Set temp dataframe to be the PCODES dataframe for the PROV
            # Only one PROV at the time
            df_temp = df_pc[df_pc['ADM2_ES'].str.upper() == prov_name]
            
            # Get the PCODE for DISTRICT by reading the DIST name in temp dataframe
            print(f'Doing: {prov_name} - {dist_name}')
            idx = df_temp.index[df_temp.ADM3_ES.str.upper() == dist_name].to_list()
            pcode = df_temp.loc[idx, 'ADM3_PCODE'].values[0]
            
            print('--------------------------------')
            
            # Add PCODE to DISTRICT
            df_final.loc[index, 'ADM3_PCODE'] = pcode

## FOR HOUSING
df_final['HOUSE_DEST'] = df_final['HOUSE_DEST'].fillna(0)
df_final['HOUSE_INHB'] = df_final['HOUSE_INHB'].fillna(0)
df_final['HOUSE_AFFT'] = df_final['HOUSE_AFFT'].fillna(0)

Doing: CHICLAYO - CAYALTI
--------------------------------
Doing: CHICLAYO - CHICLAYO
--------------------------------
Doing: CHICLAYO - CHONGOYAPE
--------------------------------
Doing: CHICLAYO - ETEN
--------------------------------
Doing: CHICLAYO - ETEN PUERTO
--------------------------------
Doing: CHICLAYO - JOSE LEONARDO ORTIZ
--------------------------------
Doing: CHICLAYO - LAGUNAS
--------------------------------
Doing: CHICLAYO - LA VICTORIA
--------------------------------
Doing: CHICLAYO - MONSEFU
--------------------------------
Doing: CHICLAYO - NUEVA ARICA
--------------------------------
Doing: CHICLAYO - OYOTUN
--------------------------------
Doing: CHICLAYO - PATAPO
--------------------------------
Doing: CHICLAYO - PICSI
--------------------------------
Doing: CHICLAYO - PIMENTEL
--------------------------------
Doing: CHICLAYO - POMALCA
--------------------------------
Doing: CHICLAYO - PUCALA
--------------------------------
Doing: CHICLAYO - REQUE
-----------

## Creating the final file for admn 2


In [19]:
df_final_adm2 = df_pop.copy()

# Remove DIST as that isn't needed anymore
df_final_adm2 = df_final_adm2[~df_final_adm2['ADM_TYPE'].isin(['DIST'])]

# Add PCODE
for index, row in df_final_adm2.iterrows():
    dist_name = row['ADM_NAME']
    print(dist_name)
    idx = df_pc.index[df_pc.ADM2_ES.str.upper() == dist_name].to_list()
    pcode = df_pc.loc[idx, 'ADM2_PCODE'].values[0]
    # print(dist_name, pcode)
    df_final_adm2.loc[index, 'ADM3_PCODE'] = pcode


## FOR HOUSING
col_names = ['ADM_NAME', 'HOUSE_DEST', 'HOUSE_INHB', 'HOUSE_AFFT', 'ADM_TYPE', 'AMD1_ES', 'AMD2_ES', 'ADM2_PCODE']
df_final_adm2.columns = col_names
df_final_adm2['HOUSE_DEST'] = df_final_adm2['HOUSE_DEST'].fillna(0)
df_final_adm2['HOUSE_INHB'] = df_final_adm2['HOUSE_INHB'].fillna(0)
df_final_adm2['HOUSE_AFFT'] = df_final_adm2['HOUSE_AFFT'].fillna(0)


df_final_adm2


CHICLAYO
FERREÑAFE
LAMBAYEQUE
AYABACA
HUANCABAMBA
MORROPON
PAITA
PIURA
SECHURA
SULLANA
TALARA
CONTRALMIRANTE VILLAR
TUMBES
ZARUMILLA


,ADM_NAME,HOUSE_DEST,HOUSE_INHB,HOUSE_AFFT,ADM_TYPE,AMD1_ES,AMD2_ES,ADM2_PCODE
1230,CHICLAYO,221.0,1094.0,6706.0,PROV,LAMBAYEQUE,CHICLAYO,PE1401
1251,FERREÑAFE,180.0,435.0,4252.0,PROV,LAMBAYEQUE,FERREÑAFE,PE1402
1258,LAMBAYEQUE,2871.0,4651.0,14675.0,PROV,LAMBAYEQUE,LAMBAYEQUE,PE1403
1542,AYABACA,84.0,333.0,7455.0,PROV,PIURA,AYABACA,PE2002
1553,HUANCABAMBA,484.0,821.0,2239.0,PROV,PIURA,HUANCABAMBA,PE2003
1562,MORROPON,165.0,506.0,7824.0,PROV,PIURA,MORROPON,PE2004
1573,PAITA,12.0,656.0,3908.0,PROV,PIURA,PAITA,PE2005
1581,PIURA,308.0,1390.0,19346.0,PROV,PIURA,PIURA,PE2001
1592,SECHURA,64.0,118.0,2636.0,PROV,PIURA,SECHURA,PE2008
1599,SULLANA,82.0,232.0,3615.0,PROV,PIURA,SULLANA,PE2006


## Setting the final file

In [20]:
# Remove PROV as that isn't needed anymore for adm3
df_final = df_final[~df_final['ADM_TYPE'].isin(['PROV'])]

# Export to Excel
# FOR HOUSING
file_name = f'per_affd_houseAffected_tab_s1_indeci_reporte_danos_lluvias_{DATA_DATE}.xlsx'

with pd.ExcelWriter(Path(fldr_outp / file_name)) as writer:  
    df_final.to_excel(writer, index=False, sheet_name='hse_impactada')
    df_final_adm2.to_excel(writer, index=False, sheet_name='hse_impactada_adm2')
    

print('--------------')
print('--------------')
print('All DONE')
print('--------------')
print('--------------')




--------------
--------------
All DONE
--------------
--------------


___
___

